# NVIDIA DeepStream Hands-On

## Goals

- Config-based execution (gst-launch, deepstream-app)
- nvinferserver
- Major plugins & deepstream-test-1-app

In [ ]:
!cd /opt/nvidia/deepstream/deepstream-6.1/samples && bash prepare_ds_triton_model_repo.sh

In [ ]:
!gst-inspect-1.0 nvinferserver

In [ ]:
!gst-launch-1.0 filesrc location=/opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.mp4 ! qtdemux ! h264parse ! nvv4l2decoder \
! m.sink_0 nvstreammux name=m batch-size=1 width=360 height=240 ! nvinfer config-file-path=/opt/nvidia/deepstream/deepstream/samples/configs/deepstream-app/config_infer_primary.txt \
batch-size=1 unique-id=1 ! nvtracker ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_nvmultiobjecttracker.so \
! nvmultistreamtiler rows=1 columns=1 width=360 height=240 ! nvvideoconvert ! nvdsosd ! nvvideoconvert ! nvv4l2h264enc ! h264parse ! qtmux ! filesink location=out.mp4

In [ ]:
from IPython.display import Video

Video("out.mp4")

<img src="img/nvinferserver_supportmatrix.png" alt="SupportMatrix" style="width: 1000px;"/>

In [ ]:
!gst-launch-1.0 filesrc location=/opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.mp4 ! qtdemux ! h264parse ! nvv4l2decoder \
! m.sink_0 nvstreammux name=m batch-size=1 width=360 height=240 \
! nvinferserver config-file-path=/opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app-triton/config_infer_primary_detector_ssd_mobilenet_v1_coco_2018_01_28.txt \
! nvtracker ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_nvmultiobjecttracker.so \
! nvmultistreamtiler rows=1 columns=1 width=360 height=240 ! nvvideoconvert ! nvdsosd ! nvvideoconvert ! nvv4l2h264enc ! h264parse ! qtmux ! filesink location=out_triton.mp4

In [ ]:
from IPython.display import Video

Video("out_triton.mp4")

In [ ]:
!cat /opt/nvidia/deepstream/deepstream/samples/configs/deepstream-app-triton/config_infer_primary_detector_ssd_mobilenet_v1_coco_2018_01_28.txt

In [ ]:
!cat /opt/nvidia/deepstream/deepstream/samples/triton_model_repo/ssd_mobilenet_v1_coco_2018_01_28/config.pbtxt

In [ ]:
!ls /opt/nvidia/deepstream/deepstream-6.1/sources/libs/nvdsinfer_customparser/

![nn](img/deepstream_app.png)

In [ ]:
!deepstream-app -c /opt/nvidia/deepstream/deepstream-6.1/workspace/deepstream_app_triton_config.txt

In [ ]:
from IPython.display import Video

Video("out_deepstream_app.mp4")

In [ ]:
!cat /opt/nvidia/deepstream/deepstream-6.1/workspace/deepstream_app_triton_config.txt

## NVIDIA DeepStream Plugins

### Nvstreammux

The Gst-nvstreammux plugin forms a batch of frames from multiple input sources.

![NVSTREAMMUX](img/nvstreammux.png)

### Nvinfer

The nvinfer plugin provides [TensorRT](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html)-based inference for detection and tracking. The lowlevel library (libnvds_infer) operates either on float RGB or BGR planar data with dimensions of Network Height and Network Width. The plugin accepts NV12/RGBA data from upstream components like the decoder, muxer, and dewarper.
The Gst-nvinfer plugin also performs preprocessing operations like format conversion, scaling, mean subtraction, and produces final float RGB/BGR planar data which is passed to the low-level library. The low-level library uses the TensorRT engine for inferencing. It outputs each classified object’s class and each detected object’s bounding boxes (Bboxes) after clustering.

![NVINFER](img/nvinfer.png)


### Nvosd

The nvosd plugin draws bounding boxes, text, and RoI (Regions of Interest) polygons (Polygons are presented as a set of lines). The plugin accepts an RGBA buffer with attached metadata from the upstream component. It
draws bounding boxes, which may be shaded depending on the configuration (e.g. width, color, and opacity) of a given bounding box. It also draws text and RoI polygons at specified locations in the frame. Text and polygon parameters are configurable through metadata.

![NVOSD](img/nvdsosd.png)

### Metadata

<img src="img/metadata.png" alt="DS Meta" style="width: 1000px;"/>

<img src="img/metadata2.png" alt="DS Meta" style="width: 1000px;"/>

<img src="img/deepstream_test1_app.png" alt="Example" style="width: 1000px;"/>

In [1]:
!cat /opt/nvidia/deepstream/deepstream-6.1/sources/apps/sample_apps/deepstream-test1/deepstream_test1_app.c

/*
 * Copyright (c) 2018-2022, NVIDIA CORPORATION. All rights reserved.
 *
 * Permission is hereby granted, free of charge, to any person obtaining a
 * copy of this software and associated documentation files (the "Software"),
 * to deal in the Software without restriction, including without limitation
 * the rights to use, copy, modify, merge, publish, distribute, sublicense,
 * and/or sell copies of the Software, and to permit persons to whom the
 * Software is furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in
 * all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL
 * THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN